In [1]:
'''Visualization of the filters of VGG16, via gradient ascent in input space.
This script can run on CPU in a few minutes.
Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]
    #loss关于iuput的梯度

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])#形成一个function f(x,y)

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])#iterate
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

Using TensorFlow backend.


58892288/58889256 [==============================] - 316s 5us/step
Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_____________________________________________________________

Current loss value: 904.912
Current loss value: 974.802
Current loss value: 1039.37
Current loss value: 1105.27
Current loss value: 1168.78
Current loss value: 1230.98
Current loss value: 1293.14
Current loss value: 1355.3
Current loss value: 1416.04
Current loss value: 1479.39
Filter 8 processed in 6s
Processing filter 9
Current loss value: 11.9718
Current loss value: 31.3635
Current loss value: 69.717
Current loss value: 126.455
Current loss value: 184.633
Current loss value: 248.157
Current loss value: 293.88
Current loss value: 360.319
Current loss value: 396.082
Current loss value: 452.03
Current loss value: 509.067
Current loss value: 562.271
Current loss value: 604.978
Current loss value: 655.587
Current loss value: 697.238
Current loss value: 729.819
Current loss value: 788.564
Current loss value: 819.188
Current loss value: 861.487
Current loss value: 888.632
Filter 9 processed in 6s
Processing filter 10
Current loss value: 9.50496
Current loss value: 21.7509
Current loss valu

Current loss value: 130.368
Current loss value: 189.908
Current loss value: 239.054
Current loss value: 299.163
Current loss value: 344.009
Current loss value: 423.35
Current loss value: 484.813
Current loss value: 561.886
Current loss value: 628.184
Current loss value: 701.035
Current loss value: 770.726
Current loss value: 861.208
Current loss value: 939.465
Current loss value: 1020.61
Filter 27 processed in 5s
Processing filter 28
Current loss value: 1.0884
Current loss value: 22.2986
Current loss value: 92.5908
Current loss value: 152.986
Current loss value: 220.648
Current loss value: 273.038
Current loss value: 320.355
Current loss value: 370.322
Current loss value: 418.896
Current loss value: 465.505
Current loss value: 508.471
Current loss value: 556.653
Current loss value: 594.625
Current loss value: 633.254
Current loss value: 674.654
Current loss value: 707.409
Current loss value: 747.327
Current loss value: 779.011
Current loss value: 825.765
Current loss value: 851.776
Fil

Current loss value: 391.772
Current loss value: 427.889
Current loss value: 463.416
Current loss value: 499.42
Current loss value: 533.658
Current loss value: 568.381
Current loss value: 600.395
Current loss value: 635.715
Current loss value: 670.518
Current loss value: 705.361
Filter 43 processed in 5s
Processing filter 44
Current loss value: 5.66297
Current loss value: 17.9877
Current loss value: 42.9258
Current loss value: 100.934
Current loss value: 156.463
Current loss value: 213.189
Current loss value: 257.908
Current loss value: 300.195
Current loss value: 340.241
Current loss value: 381.319
Current loss value: 411.732
Current loss value: 451.796
Current loss value: 483.825
Current loss value: 514.944
Current loss value: 548.419
Current loss value: 580.374
Current loss value: 616.822
Current loss value: 651.769
Current loss value: 683.597
Current loss value: 723.027
Filter 44 processed in 5s
Processing filter 45
Current loss value: 0.051973
Current loss value: 15.1959
Current lo

...

Current loss value: 312.285
Current loss value: 353.074
Current loss value: 392.541
Current loss value: 427.855
Current loss value: 463.644
Current loss value: 494.623
Current loss value: 531.454
Current loss value: 558.826
Current loss value: 595.838
Current loss value: 628.462
Current loss value: 662.015
Filter 121 processed in 6s
Processing filter 122
Current loss value: 0.578646
Current loss value: 6.78713
Current loss value: 39.2544
Current loss value: 75.9189
Current loss value: 111.273
Current loss value: 149.76
Current loss value: 174.787
Current loss value: 215.915
Current loss value: 263.499
Current loss value: 299.7
Current loss value: 334.509
Current loss value: 373.032
Current loss value: 415.987
Current loss value: 459.75
Current loss value: 505.985
Current loss value: 548.922
Current loss value: 591.513
Current loss value: 632.791
Current loss value: 673.941
Current loss value: 717.31
Filter 122 processed in 6s
Processing filter 123
Current loss value: 0.461386
Current l

Current loss value: 518.376
Current loss value: 550.256
Current loss value: 588.959
Current loss value: 627.869
Current loss value: 665.505
Current loss value: 698.745
Current loss value: 740.849
Current loss value: 777.167
Filter 137 processed in 6s
Processing filter 138
Current loss value: 40.0037
Current loss value: 231.277
Current loss value: 382.755
Current loss value: 501.317
Current loss value: 602.155
Current loss value: 694.795
Current loss value: 796.472
Current loss value: 898.634
Current loss value: 994.512
Current loss value: 1091.74
Current loss value: 1178.75
Current loss value: 1266.17
Current loss value: 1354.07
Current loss value: 1443.56
Current loss value: 1531.05
Current loss value: 1622.29
Current loss value: 1711.84
Current loss value: 1802.83
Current loss value: 1886.2
Current loss value: 1974.55
Filter 138 processed in 6s
Processing filter 139
Current loss value: 12.4191
Current loss value: 40.9037
Current loss value: 75.1264
Current loss value: 125.812
Current

Current loss value: 473.46
Current loss value: 506.683
Current loss value: 549.301
Current loss value: 600.542
Current loss value: 641.442
Filter 153 processed in 6s
Processing filter 154
Current loss value: 18.3242
Current loss value: 80.8684
Current loss value: 150.471
Current loss value: 223.771
Current loss value: 282.107
Current loss value: 345.214
Current loss value: 412.575
Current loss value: 477.434
Current loss value: 528.502
Current loss value: 578.465
Current loss value: 637.786
Current loss value: 693.798
Current loss value: 745.869
Current loss value: 795.632
Current loss value: 848.806
Current loss value: 896.895
Current loss value: 938.501
Current loss value: 985.782
Current loss value: 1028.88
Current loss value: 1077.77
Filter 154 processed in 6s
Processing filter 155
Current loss value: 13.5393
Current loss value: 34.8765
Current loss value: 56.081
Current loss value: 78.9205
Current loss value: 101.606
Current loss value: 140.771
Current loss value: 163.276
Current 

Current loss value: 742.065
Current loss value: 789.182
Current loss value: 848.859
Current loss value: 899.503
Filter 170 processed in 6s
Processing filter 171
Current loss value: 46.3599
Current loss value: 112.234
Current loss value: 179.188
Current loss value: 257.038
Current loss value: 332.015
Current loss value: 394.811
Current loss value: 456.18
Current loss value: 495.272
Current loss value: 556.724
Current loss value: 615.575
Current loss value: 664.353
Current loss value: 708.121
Current loss value: 761.889
Current loss value: 799.713
Current loss value: 837.755
Current loss value: 883.91
Current loss value: 917.995
Current loss value: 971.637
Current loss value: 1012.0
Current loss value: 1063.09
Filter 171 processed in 6s
Processing filter 172
Current loss value: 0.0
Filter 172 processed in 0s
Processing filter 173
Current loss value: 1.55552
Current loss value: 27.7414
Current loss value: 85.9149
Current loss value: 175.825
Current loss value: 266.483
Current loss value: 

Current loss value: 1556.01
Current loss value: 1626.75
Current loss value: 1696.15
Filter 187 processed in 6s
Processing filter 188
Current loss value: 2.76484
Current loss value: 34.3877
Current loss value: 94.6564
Current loss value: 143.953
Current loss value: 202.895
Current loss value: 257.153
Current loss value: 316.056
Current loss value: 376.255
Current loss value: 422.339
Current loss value: 473.186
Current loss value: 518.215
Current loss value: 558.607
Current loss value: 600.183
Current loss value: 644.861
Current loss value: 691.816
Current loss value: 731.457
Current loss value: 775.994
Current loss value: 818.669
Current loss value: 854.358
Current loss value: 892.201
Filter 188 processed in 6s
Processing filter 189
Current loss value: 0.372409
Current loss value: 2.54523
Current loss value: 16.0789
Current loss value: 48.3336
Current loss value: 88.7537
Current loss value: 133.458
Current loss value: 185.007
Current loss value: 228.009
Current loss value: 273.857
Curre